In [9]:
import numpy as np
from warmup_scheduler import GradualWarmupScheduler
import pandas as pd
import os
import torch
from torch import nn, optim
from sklearn.model_selection import train_test_split

from work.utils.dataset import PandasDataset
from work.utils.dataset import RemovePenMarkAlbumentations
import albumentations as A

from torch.utils.data import DataLoader
from work.utils.models import EfficientNet
from work.utils.train import train_model
from work.utils.metrics import model_checkpoint

In [2]:
backbone_model = 'efficientnet-b0'
pretrained_model = {
    backbone_model: 'pre-trained-models/efficientnet-b0-08094119.pth'
}
data_dir = 'data'
images_dir = os.path.join(data_dir, 'tiles')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [10]:
# Leitura dos dados
df_remove = pd.read_csv(f"{data_dir}/remove-images.csv").sort_values(by=["entropy"], ascending=False)
df = pd.read_csv(f"{data_dir}/train_val.csv")
df_remove_filtered = df_remove[df_remove["entropy"] < 1.3]
#
df_filtered = df[df["image_id"].isin(df_remove_filtered["image_id"])]
print(f"Original shape: {df.shape}")
print(f"Filtered shape: {df_filtered.shape}")

array([1.3160478, 1.3184186, 1.3194076, 1.3210733, 1.3219315])

In [4]:
batch_size = 2
num_workers = 4
output_classes = 5
init_lr = 3e-4
loss_function = nn.BCEWithLogitsLoss()
epochs = 50
n_folds = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
transforms = A.Compose([
    # RemovePenMarkAlbumentations(),
    A.Transpose(p=0.5),
    A.VerticalFlip(p=0.5),
    A.HorizontalFlip(p=0.5),
])

In [6]:
df_train, df_val = train_test_split(df_filtered, test_size=0.20)

dataset_train = PandasDataset("../dataset/tiles", df_train, transforms=transforms)
dataset_valid = PandasDataset("../dataset/tiles", df_val)

print(f"train: {len(dataset_train)} images | validation: {len(dataset_valid)} images ")

train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, num_workers=num_workers)

valid_loader = DataLoader(dataset_valid, batch_size=batch_size, shuffle=True, num_workers=num_workers)

model = EfficientNet(backbone_model, output_classes, weights_path=pretrained_model.get(backbone_model))
optimizer = optim.Adam(model.parameters(), lr=init_lr)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs - 1)

scheduler = GradualWarmupScheduler(optimizer, multiplier=10, total_epoch = 1, after_scheduler=scheduler_cosine)


save_path = f'pre-trained-models/removed-images.pth'

train_model(
    model,
    epochs,
    optimizer,
    scheduler,
    train_loader,
    valid_loader,
    df_val,
    checkpoint=model_checkpoint,
    device=device,
    loss_function=loss_function,
    path_to_save_metrics="logs/history/removed-images.txt",
    path_to_save_model=save_path,
)



train: 6433 images | validation: 1609 images 
Loaded pretrained weights for efficientnet-b0
Epoch 1/50



100%|██████████| 805/805 [01:48<00:00,  7.44it/s]


Epoch 2/50



100%|██████████| 805/805 [01:48<00:00,  7.42it/s]


Epoch 3/50



...

100%|██████████| 805/805 [01:47<00:00,  7.46it/s]


Epoch 49/50



100%|██████████| 805/805 [01:47<00:00,  7.46it/s]


Epoch 50/50



100%|██████████| 805/805 [01:48<00:00,  7.45it/s]


In [9]:
model_checkpoint(model, 9, 10, save_path)

Salvando o melhor modelo... 9 -> 10


10